In [ ]:
import pyawr.mwoffice as mwo

import numpy as np
from numpy.polynomial.polynomial import polyfit

import matplotlib.pyplot as plt

import qkit.analysis.circle_fit.circle_fit_2019.circuit as cf

#***PICK ONE***
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
# %matplotlib tk 

In [ ]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #In/voke circuit simulator analysis

In [ ]:
def reset_freqs (l_bnd=4e9, u_bnd=8e9, steps=10000):
    '''
    Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.
    '''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return freq_arr

In [ ]:
def set_circ_params(circ_name='Sample_Subcircuit', get_params=False, **kwargs): 
    '''Sets specified circuit parameters in circuit called circ_name.
    

    If you attempt to pass a parameter that doesn't exit in the circuit called circ_name, 
    MWO will throw an error.

    Returns a dictionary with the new values.
    '''

    passed_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2).ValueAsDouble,
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2).ValueAsDouble,
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2).ValueAsDouble
    }
    passed_circ_params = { # dictionary of subcircuit component 
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2)
    }

    # print("DEBUG: Setting:" + kwargs.__str__())

    new_circ_param_vals = {**passed_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    # print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(passed_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(passed_circ_param_vals)[i] + " set to " + str(value))
    
    if not(get_params):
        awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return new_circ_param_vals

In [ ]:
def create_polar_graph(schem_name='Hanger_Test'):
    graphs = awrde.Project.Graphs
    # TODO: Name helper graph appropriately
    graph = graphs.Add("CalcHelper {}".format(schem_name), mwo.mwGraphType.mwGT_Polar)

    graph.Measurements.Add(schem_name, 'S(2,1)')

    return graph

In [ ]:
def delete_graph(graph=None):
    for i in range(awrde.Project.Graphs.Count):
        if awrde.Project.Graphs.Item(i+1).Name == graph.Name:
            ret = awrde.Project.Graphs.Remove(i+1)

    return ret

In [ ]:
def get_meas_vals(graph=None):
    #TODO: iterate over all measurement indicies, add a try-catch for when there's only 1 measurement on the graph
    meas = graph.Measurements[0]

    trace = meas.TraceValues(1)
    freqs, real, imag = zip(*trace)

    return (freqs, real, imag)

In [ ]:
def plot_polar(re, im):
    print(re)
    print(im)

    plt.scatter(re, im)

In [ ]:
gr = create_polar_graph()
awrde.Project.Simulator.Analyze()
meas_vals = get_meas_vals(gr)
freqs, real, imag = np.asarray(meas_vals)
plot_polar(real, imag)
cpx = real + imag*1j

In [ ]:
delete_graph(gr)

In [ ]:
def do_stuff():

    gr = create_polar_graph()
    awrde.Project.Simulator.Analyze()
    meas_vals = get_meas_vals(gr)
    freqs, real, imag = np.asarray(meas_vals)
    plot_polar(real, imag)
    cpx = real + imag*1j
    delete_graph(gr)

    circ = cf.reflection_port(freqs, cpx)
    circ.autofit()
    circ.plotall()
    print(circ.fitresults)

    return circ

In [ ]:
def is_fit_good(circ):
    fit_good = False

    if circ.Ql>0 and circ.Qc>0 and circ.Qi>0:
        print("Qs all positive.")

        if circ.f_data[0] < circ.fr < circ.f_data[-1]:
            print("fr in f_data")
            fit_good = True
        
        else:
            fit_good = False

    else:
        fit_good = False
    
    return fit_good

In [ ]:
reset_freqs()
ds = do_stuff()
is_fit_good(ds)

reset_freqs(3e9, 9e9, 10000)
ds = do_stuff()
is_fit_good(ds)

reset_freqs(3.4e9, 5e9, 10000)
ds = do_stuff()
is_fit_good(ds)

reset_freqs(7e9, 8.6e9, 10000)
ds = do_stuff()
is_fit_good(ds)


In [ ]:
from sympy import Symbol, diff, exp, cos, pi, lambdify

class analyze_circuit(cf.circuit):
    '''
    Circlefit class for finding the exact derivative of Sij and maximizing it.
    '''

    n_ports = None
    f_data = None

    def __init__(self, circ):
        self.n_ports = circ.n_ports
        self.f_data = circ.f_data

    def sensitive_Sij(self, fr, Ql, Qc, phi=0., a=1., alpha=0., delay=0.):
        '''
        Returns the frequency at which the magnitude of the derivative of Sij is maximized.
        '''

        f = Symbol('f', real=True)

        complexQc = Qc*cos(phi)*exp(-1j*phi)
        S = a*exp(1j*(alpha-2*pi*f*delay)) * (
            1. - 2.*Ql / (complexQc * self.n_ports * (1. + 2j*Ql*(f/fr-1.)))
        )

        ds = diff(S, f)
        ds_np = lambdify(f, ds, 'numpy')
        #NOTE: I'm just maximizing the magnititude of the derivatives of Sij here... Complex numbers have no ordering
        ds_mags = np.asarray(np.abs(ds_np(self.f_data)))
        arg = np.argmax(ds_mags)

        return self.f_data[arg]




In [ ]:
new_circ = analyze_circuit(ds)
max = new_circ.sensitive_Sij(ds.fr, ds.Ql, ds.Qc, ds.phi, ds.a, ds.alpha, ds.delay)

In [ ]:
def fit_circ(circ_type='reflection_port'):

    gr = create_polar_graph()
    awrde.Project.Simulator.Analyze()
    meas_vals = get_meas_vals(gr)
    freqs, real, imag = np.asarray(meas_vals)
    plot_polar(real, imag)
    cpx = real + imag*1j
    delete_graph(gr)

    if circ_type == 'reflection_port':
        circ = cf.reflection_port(freqs, cpx)
        circ.autofit()
        circ.plotall()
        print(circ.fitresults)
    
    elif circ_type == 'notch_port':
        circ = cf.notch_port(freqs, cpx)
        circ.autofit()
        circ.plotall()
        print(circ.fitresults)

    else:
        print("Invalid circuit measurement mode! Must be 'reflection' or 'notch'.")

    return circ

In [ ]:
def bin_search(freq_arr, circ_type='reflection_port', debug=False):
    steps = 10000

    def left_search():

        if debug:
            print('left search')

        l_bnd = freq_arr[0]
        u_bnd = freq_arr[np.shape(freq_arr)[0]//2]

        reset_freqs(l_bnd, u_bnd, steps)
        circ = fit_circ(circ_type)

        return circ

    def right_search():

        if debug:
            print('right search')

        l_bnd = freq_arr[np.shape(freq_arr)[0]//2]
        u_bnd = freq_arr[-1]
        
        reset_freqs(l_bnd, u_bnd, steps)
        circ = fit_circ(circ_type)

        return circ

    def check_fit(circ):
        if debug:
            print('frequency bounds: ' + (freq_arr[0], freq_arr[-1], steps).__str__())
            print('resonant freqency: ' + str(circ.fr))
        
        if is_fit_good(circ):
            three_db_bw = circ.fr/circ.Ql

            if 3.5 * three_db_bw < freq_arr[-1] - freq_arr[0] < 4.5 * three_db_bw:
                if debug:
                   print('3.5 * three_db_bw ({}) < freq_arr[-1] ({}) - freq_arr[0] ({}) < 4.5 * three_db_bw ({})'.format(3.5*three_db_bw, freq_arr[-1] - freq_arr[0]), 4.5 * three_db_bw) 
                
                return is_fit_good(circ) and circ.f_data[0] < circ.fr < circ.f_data[-1]

            elif 3.5 * three_db_bw >= freq_arr[-1] - freq_arr[0]:
                if freq_arr[0] - 5e8 > 0:    
                    reset_freqs(freq_arr[0] - 5e8, freq_arr[-1] + 1e9)

            elif freq_arr[-1] - freq_arr[0] <= 5e8:
                if debug:
                    print('')
                    print('Pruned branch.')

                return False
        
        else:
            return False
    
    l_test_circ = left_search()
    r_test_circ = right_search()
    circs = (l_test_circ, r_test_circ)

    for ci in circs:
        if check_fit(ci):
            new_circ = analyze_circuit(ci)
            max = new_circ.sensitive_Sij(ci.fr, ci.Ql, ci.Qc, ci.phi, ci.a, ci.alpha, ci.delay)

            return max
        
        else:
            if debug:
                print('Searching: bin_search' + (ci.f_data, circ_type, debug).__str__())
                
            bin_search(ci.f_data, circ_type, debug)

In [ ]:
def search_ideal(circ, circ_type='reflection_port', debug=False):
    steps = 10000
    new_freqs = circ.f_data
    circuit_type = circ_type
    current_circuit = circ

    def move_left(incr = 5e8):        
        if debug:
            print('Moving left.')
            
        if new_freqs[0] - incr >= 0:
            current_circuit = fit_circ(circuit_type)
            return reset_freqs(new_freqs[0] - incr, new_freqs[-1] - incr)

    def move_right(incr = 5e8):
        if debug:
            print('Moving right.')

        if new_freqs[-1] <= 10e9:
            current_circuit = fit_circ(circuit_type)
            return reset_freqs(new_freqs[0] + incr, new_freqs[-1] + incr)
    
    def split(circ):
        if debug:
            print('Splitting.')

        l_bnd = new_freqs[0]
        u_bnd = new_freqs[np.shape(new_freqs)[0]//2]
        reset_freqs(l_bnd, u_bnd)
        v1 = search_ideal(fit_circ(circuit_type), circuit_type, debug)
        l_bnd = new_freqs[np.shape(new_freqs)[0]//2]
        u_bnd = new_freqs[-1]
        reset_freqs(l_bnd, u_bnd)
        v2 = search_ideal(fit_circ(circuit_type), circuit_type, debug)

        return max(v1, v2)
    
    def fine_tune(circ):
        if debug:
            print('Fine tuning.')

        current_circuit = circ
        current_circuit.autofit()
        cc_fr_err = current_circuit.fitresults.get('fr_err')
        move_left()
        current_circuit.autofit()

        if cc_fr_err < current_circuit.fitresults.get('fr_err'):
            move_right()
            move_right()
            current_circuit.autofit()

            if cc_fr_err < current_circuit.fitresults.get('fr_err'):
                move_left()
                return current_circuit


        return fine_tune(current_circuit)
        
    def expand(incr = 5e8):
        if debug:
            print('Expanding.')
        if new_freqs[-1] + incr <= 10e9:
            new_freqs = reset_freqs(new_freqs[0], new_freqs[-1] + incr)
        if freq_arr[0] + incr >= 0:
            new_freqs = reset_freqs(new_freqs[0] - incr, new_freqs[-1])
            
        current_circuit = fit_circ(circ, circuit_type)
        return new_freqs

    def contract(incr = 5e8):
        if debug:
            print('Contracting.')
        if new_freqs[-1] - new_freqs[0] > 3*incr:
            new_freqs = reset_freqs(new_freqs[0] + incr, new_freqs[-1] - incr)
            current_circuit = fit_circ(circuit_type)
            return new_freqs

    def check_roll_off():
        phase = np.unwrap(np.angle(circ.z_data))
        if np.max(phase) - np.min(phase) <= 0.8*2*np.pi:
            if debug:
                print('Roll-off is less than ~2pi. Expanding bounds.')
            
            expand()
            return check_roll_off()
        
        else:
            return new_freqs

    def check_bw():
        three_db_bw = circ.fr/circ.Ql

        if debug:
            print('checking bandwidth.')
            text = f'3.5 * three_db_bw ({3.5*three_db_bw}) < new_freqs[-1] ({new_freqs[-1]}) - new_freqs[0] ({new_freqs[0]}) < 4.5 * three_db_bw ({4.5 * three_db_bw})'
            print(text)
        
        if new_freqs[-1] - new_freqs[0] > 5.*three_db_bw:
            contract()
            return check_roll_off()
        elif new_freqs[-1] - new_freqs[0] < 3.*three_db_bw:
            expand()
            return check_roll_off()
        else:
            return new_freqs
        
    def get_max_deriv(circ):
        new_circ = analyze_circuit(circ)
        max = new_circ.sensitive_Sij(circ.fr, circ.Ql, circ.Qc, circ.phi, circ.a, circ.alpha, circ.delay)
        return max

    def check_fit(circ):
        if debug:
            print('frequency bounds: ' + (new_freqs[0], new_freqs[-1], steps).__str__())
            print('resonant freqency: ' + str(circ.fr))
        
        check_bw()

        if is_fit_good(circ):
            return fine_tune(circ)
        else:
            return split(circ)
    
    return get_max_deriv(check_fit(circ))

In [ ]:
reset_freqs(2e9, 12e9)
circuit = fit_circ('reflection_port')
search_ideal(circuit, debug=True)